In [1]:
import pandas as pd
import numpy as np
import re
import sys
import json
import requests
from lxml import etree
from IPython.display import display, HTML
import urllib
#from bs4 import BeautifulSoup as Soup

df_prefix = "/mnt/d/OneDrive/OnedriveDocuments/NTU/Work/thesis/dataset(2004-2016)/"
df = pd.read_excel(df_prefix+"legislator_additional_attributes.xlsx")

legislators_ethicity_reference = {}
legislators_ethicity_jsonfile = df_prefix+'legislators_ethicity.txt'
legislators_ethicity_reference_jsonfile = df_prefix+'legislators_ethicity_reference.txt'
jquery_file = df_prefix+'jquery-3.3.1.min.js'
bulletin_urls = ["0Zero",
    "1One",
    "http://ebulletin.cec.gov.tw/%E9%81%B8%E8%88%89/%E7%AB%8B%E6%B3%95%E5%A7%94%E5%93%A1/081%E5%B9%B4%E7%AC%AC2%E5%B1%86/",
    "http://ebulletin.cec.gov.tw/%E9%81%B8%E8%88%89/%E7%AB%8B%E6%B3%95%E5%A7%94%E5%93%A1/084%E5%B9%B4%E7%AC%AC3%E5%B1%86/index2.php",
    "http://ebulletin.cec.gov.tw/%E9%81%B8%E8%88%89/%E7%AB%8B%E6%B3%95%E5%A7%94%E5%93%A1/087%E5%B9%B4%E7%AC%AC4%E5%B1%86/index2.php",
    "http://ebulletin.cec.gov.tw/%E9%81%B8%E8%88%89/%E7%AB%8B%E6%B3%95%E5%A7%94%E5%93%A1/090%E5%B9%B4%E7%AC%AC5%E5%B1%86/index2.php",
    "http://ebulletin.cec.gov.tw/%E9%81%B8%E8%88%89/%E7%AB%8B%E6%B3%95%E5%A7%94%E5%93%A1/093%E5%B9%B4%E7%AC%AC6%E5%B1%86/index2.php",
    "http://ebulletin.cec.gov.tw/%E9%81%B8%E8%88%89/%E7%AB%8B%E6%B3%95%E5%A7%94%E5%93%A1/097%E5%B9%B4%E7%AC%AC7%E5%B1%86/index2.php",
    "http://ebulletin.cec.gov.tw/%E9%81%B8%E8%88%89/%E7%AB%8B%E6%B3%95%E5%A7%94%E5%93%A1/101%E5%B9%B4%E7%AC%AC8%E5%B1%86/index2.php",
    "http://ebulletin.cec.gov.tw/%E9%81%B8%E8%88%89/%E7%AB%8B%E6%B3%95%E5%A7%94%E5%93%A1/105%E5%B9%B4%E7%AC%AC9%E5%B1%86/"]
urlencodedquote = urllib.parse.quote_plus("\"")
def addlegislatoreth(name,key,dic):
    choices = {
        'h': 'hakka',
        'w': 'foreignstates',
        'f': 'fulo',
        'u': 'unknown',
        'o': 'other',
        'n': 'newresident',
        'a': 'aboriginal'
    }
    result = choices.get(key.lower(), 'unknown')
    dic[result] = "|".join((dic[result], name))
    return(dic)
def generate_hyperlink_for_url(url, text_for_link=""):
    if (text_for_link==""):
        text_for_link = url
    linktext = '<a href="'+url+'">'+text_for_link+'</a>'
    return(linktext)

path_to_candidate_api = 'http://db.cec.gov.tw/histCand.jsp'
path_to_legislatorrelatives_api = 'https://opentaipower.g0v.tw/relatives'
response_legislatorrelatives_data = requests.get(path_to_legislatorrelatives_api, data={})
legislatorrelatives_rootselector = etree.HTML(response_legislatorrelatives_data.text)
relative_for_a_legislator_inf_search_xpathkey = {"relationship":".//td[2]","name":".//td[1]","groupname":".//td[3]","grouppos":".//div[4]"}

request_candidate_header = {
    "accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
    "accept-language":"zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
    "cache-control":"max-age=0",
    "content-type":"application/x-www-form-urlencoded",
    "upgrade-insecure-requests":"1",
    "referrer":"http://db.cec.gov.tw/histCand.jsp",
    "referrerPolicy":"no-referrer-when-downgrade"
}
election_inf_search_xpathkey = {"electiondate":".//div[1]","party":".//div[4]","wonelection":".//div[5]","area":".//div[6]","electionname":".//div[7]/a"}

usingselenium = False
#sys.exit()

if (usingselenium):
    # Create a desired capabilities object as a starting point.
    import webbrowser as webb
    from selenium import webdriver
    from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.common.keys import Keys
    chrome_options = Options()
    chrome_options.add_argument("--incognito")
    chrome_options.add_argument('--window-size=1360,720')
    capabilities = {**DesiredCapabilities.CHROME.copy(), **chrome_options.to_capabilities()}
    print(capabilities)

In [ ]:
if (usingselenium):
    # Instantiate an instance of Remote WebDriver with the desired capabilities.
    driver = webdriver.Remote(desired_capabilities=capabilities,
                              command_executor="http://localhost:4445/wd/hub")
    driver.get('http://books.google.com.tw')
    c = driver.find_element_by_tag_name("body")
    c.send_keys(Keys.LEFT_CONTROL+Keys.ADD)
    c.send_keys(Keys.LEFT_CONTROL+Keys.ADD)
    c.send_keys(Keys.LEFT_CONTROL+Keys.ADD)
    driver.execute_script("$('body').css('transform', 'scale(1.2)');")
    driver.execute_script('document.body.style.fontSize = ((window.innerHeight/100)*30)+"%"')
    with open(jquery_file, 'r') as jquery_js: 
        jquery = jquery_js.read()
        driver.execute_script(jquery)
        driver.execute_script('$("body").css("font-size", "150%")')

In [6]:
legisnames = pd.unique(df['legislator_name'])
for legisname in legisnames:
    position = list(legisnames).index(legisname)
    all_num_elements = len(legisnames)
    restnumelements = all_num_elements-position
    servingterms = (df.loc[(df['legislator_name']==legisname),['term']])['term'].tolist()
    ethnicity_in_excel = pd.unique((df.loc[(df['legislator_name']==legisname),["legislator_ethnicity"]])["legislator_ethnicity"]).tolist()[0]
    with open(legislators_ethicity_jsonfile, 'r') as pickle_file:
        try:
            legislators_ethicity = json.load(pickle_file)
        except:
            legislators_ethicity = {
                'aboriginal':'',
                'foreignstates':'尹伶瑛|段宜康|趙麗雲|吳育昇|丁守中|朱鳳芝|周守訓|邱毅|帥化民|洪秀柱|孫大千|李慶安|李慶華|潘維剛|蔣孝嚴|賴士葆|費鴻泰|盧秀燕|王榮璋|顧立雄|段宜康|王定宇|趙天麟|梁文傑|王鍾渝|李永萍|江綺雯|沈智慧',
                'fulo':'尤清|王拓|王金平|王幸男|王政中|王昱婷|王雪峰|江丙坤|江昭儀|何金松|何敏豪|余政道|吳東昇|吳敦義|呂新民|李文忠|李全教|蔡正元|李嘉進|林豐正|郭素春|邱毅|李明憲|李俊毅|李鴻鈞|杜文卿|沈富雄|邱永仁',
                'hakka':'羅文嘉|林郁方|羅志明|彭添富|邱垂貞|張昌財|邱創良|鄭金玲|張學舜|邱鏡淳|陳進興|呂學樟|何智輝|徐耀昌|林豐喜|邱太三|郭俊銘|鍾紹和|傅崐萁|饒穎奇|李桐豪|鍾榮吉|徐中雄|吳志揚|彭紹瑾|鄭金玲|葉芳雄|管碧玲|張慶惠|劉盛良|廖正井|趙麗雲|邱志偉|呂玉玲|徐欣瑩|邱文彥|陳碧涵|吳宜臻|李應元|陳賴素美|徐志榮|鍾佳濱|鍾孔炤|林為洲|陳明真',
                'newresident':'',
                'other':'吳成典'
                #'unknown':'' 王淑慧|朱星羽|李和順|李雅景|李鎮楠|李顯榮|李雅景|邱創進
            } #infile unknown: 朱星羽|李和順|李雅景|李鎮楠|李顯榮|李雅景|邱創進|邱彰|卓榮泰|周伯倫|周慧瑛|林育生|林政義|柯淑敏|鄭三元|謝明源|梅長錡|李貴敏|曾巨威|詹滿容|陳曼麗
        finally:
            pickle_file.close()

    with open(legislators_ethicity_reference_jsonfile, 'r') as pickle_file:
        legislators_ethicity_reference = json.load(pickle_file)
        pickle_file.close()

    legislators_ethicity_str = "|".join(legislators_ethicity.values())

    pattern = re.compile(legisname)
    # for key in ethnicity mode
    if (  (pattern.search(legislators_ethicity_str)!=None) | (isinstance(ethnicity_in_excel,str))  ):
        #print(legisname+" passed!")
        continue
    # for key in reference mode
    
    s1 = ""
    s2 = "+"
    googlebooksearchurl_fs=s1.join( ('https://www.google.com.tw/search?hl=zh-TW&tbo=p&tbm=bks&num=100&q=', urlencodedquote, '外省', urlencodedquote, s2, legisname ) )
    googlebooksearchurl_hakka=s1.join( ('https://www.google.com.tw/search?hl=zh-TW&tbo=p&tbm=bks&num=100&q=', urlencodedquote, '客家', urlencodedquote, s2, legisname ) )
    googlebooksearchurl_fulo=s1.join( ('https://www.google.com.tw/search?hl=zh-TW&tbo=p&tbm=bks&num=100&q=', urlencodedquote, '閩南', urlencodedquote, s2, legisname ) )
    wikipage=s1.join( ('http://zh.wikipedia.org/wiki/', legisname) )
    lysearch=s1.join( ('https://www.google.com.tw/search?&num=100&q=site:ly.gov.tw+立法委員+', urlencodedquote, legisname, urlencodedquote) )
    mothersearch=s1.join( ('https://www.google.com.tw/search?num=100&q=立法委員+', urlencodedquote, legisname, urlencodedquote, "+母親") )
    fathersearch=s1.join( ('https://www.google.com.tw/search?num=100&q=立法委員+', urlencodedquote, legisname, urlencodedquote, "+父親") )
    parentssearch=s1.join( ('https://www.google.com.tw/search?num=100&dcr=0&q=父親+母親+立法委員+爸媽+', urlencodedquote, legisname, urlencodedquote) )
    ytsearch=s1.join( ('https://www.youtube.com/results?search_type=search_videos&search_sort=relevance&search_category=0&page=&search_query=', legisname) )
    votenccu=s1.join( ('https://www.google.com/search?ie=UTF-8&q=site%3Avote.nccu.edu.tw+', urlencodedquote, legisname, urlencodedquote) )
    #暗公報 https://election.thenewslens.com/2018/html/article-1-6-3-taichung.html
    print(legisname + "( " + str(restnumelements) + " to go )")
    #print((df.loc[(df['name']==legisname),['term']])['term'])
    #print(servingterms)
    #sys.exit()
    displayurllist = [wikipage,lysearch,googlebooksearchurl_fs,googlebooksearchurl_hakka,googlebooksearchurl_fulo,parentssearch,mothersearch,fathersearch,ytsearch,votenccu]
    displayurllist += list(np.take(bulletin_urls,servingterms))
    for url in displayurllist:
        #print(url)
        display(HTML(generate_hyperlink_for_url(url)))
        if (usingselenium):
            driver.get(url)
            with open(jquery_file, 'r') as jquery_js: 
                jquery = jquery_js.read()
                driver.execute_script(jquery)
                driver.execute_script('$("body").css("font-size", "150%")')
                driver.execute_script('$("div.rc").css("font-size", "150%")')
                driver.execute_script('$("div.mw-parser-output").css("font-size", "125%")')
                driver.execute_script('$("table.infobox").css("font-size", "large")')
                finishonepage=input("Press Enter to continue...")
                if (finishonepage=='end'):
                    break
    relative_datadiv_for_a_legislator_trs = legislatorrelatives_rootselector.xpath("//div[@class='card-body' and contains(.,'"+legisname+"')]//tr")[1:]
    display("total number of relatives in g0v data are:"+str(len(relative_datadiv_for_a_legislator_trs)))
    search_i = 1
    try:
        for relative_of_a_legislator in relative_datadiv_for_a_legislator_trs:
            str_search_i = str(search_i)
            for td_search_key,td_search_value in relative_for_a_legislator_inf_search_xpathkey.items():
                try:
                    match_relative_result = relative_of_a_legislator.find(td_search_value).text.strip()
                    if (td_search_key=="name"):
                        search_relative_url = "https://www.google.com.tw/search?num=100&q="+match_relative_result
                        str_search_i += generate_hyperlink_for_url(search_relative_url)+"　"
                    else:
                        str_search_i += match_relative_result+"　"
                except:
                    str_search_i += ""
            display(HTML(str_search_i))
            search_i += 1
    except:
        display("")
    request_candidate_data = {'candName':legisname}
    response_candidate_data = requests.post(path_to_candidate_api, data=request_candidate_data)#, headers=request_candidate_header
    #soup = BeautifulSoup(response_candidate_data, 'html.parser')
    rootselector = etree.HTML(response_candidate_data.text)
    participated_elections = rootselector.xpath("/html/body/div[4]/div[3]//div[contains(@style,'width: 1200px')]")
    election_inf = {}
    for participated_election in participated_elections:
        for search_i in range(1,7):
            str_search_i = str(search_i)
            try:
                election_inf[str_search_i] = participated_election.find(".//div["+str_search_i+"]").text
            except:
                election_inf[str_search_i] = ""
        try:
            electionlink = "http://db.cec.gov.tw/"+participated_election.find(".//div[7]/a").attrib.get("href")
            electionlink = generate_hyperlink_for_url(electionlink)
            election_inf['electionlink'] = electionlink
        except:
            electionlink = ""
            election_inf['electionlink'] = electionlink
        try:
            display(HTML( str.join(' ', election_inf.values()) ))
        except:
            display("")
    category = input("Choose Category of a Legislator: H 客家 F 閩南 W 外省 A 原住民 N 新住民 O 其他 U 不明：")
    with open(legislators_ethicity_jsonfile, 'w') as pickle_file:
        legislators_ethicity = addlegislatoreth(legisname,category,legislators_ethicity)
        json.dump(legislators_ethicity, pickle_file, ensure_ascii=False)
    if (len(legislators_ethicity_reference)>1):
        legislators_ethicity_reference[legisname] = input("Category Reference：")
        items = re.findall(r'https?://([-\w./?=&+\d%\"])+',legislators_ethicity_reference[legisname])
        for item in items:
            legislators_ethicity_reference[legisname] = legislators_ethicity_reference[legisname].replace(item, urllib.parse.quote_plus(item))
        with open(legislators_ethicity_reference_jsonfile, 'w') as pickle_file:
            json.dump(legislators_ethicity_reference, pickle_file, ensure_ascii=False)
    else:
        display("error in legislators_ethicity_reference")
    #print(legisname)

瓦歷斯．貝林( 463 to go )


'total number of relatives in g0v data are:0'

Choose Category of a Legislator: H 客家 F 閩南 W 外省 A 原住民 N 新住民 O 其他 U 不明： a
Category Reference： 選舉公報和名字


林奕華( 3 to go )


'total number of relatives in g0v data are:1'

...

'total number of relatives in g0v data are:1'

Choose Category of a Legislator: H 客家 F 閩南 W 外省 A 原住民 N 新住民 O 其他 U 不明： f
Category Reference： father陳安邦's background https://zh.wikipedia.org/wiki/陳安邦 and not hakka


童惠珍( 1 to go )


'total number of relatives in g0v data are:0'

Choose Category of a Legislator: H 客家 F 閩南 W 外省 A 原住民 N 新住民 O 其他 U 不明： w
Category Reference： 平凡中的不平凡：全美台聯40風雲人物 page126 父親是江西人 http://fliphtml5.com/xxmy/dhzq/basic/101-150


In [ ]:
df